In [ ]:
# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import lifelines
import pycox
import xgboost as xgb
from sklearn.model_selection import train_test_split

from xgbse import XGBSEKaplanNeighbors, XGBSEKaplanTree, XGBSEDebiasedBCE, XGBSEBootstrapEstimator, XGBSEStackedWeibull
from xgbse.converters import convert_data_to_xgb_format, convert_to_structured, convert_y
from xgbse.non_parametric import get_time_bins

from benchmark import BenchmarkLifelines, BenchmarkXGBoost, BenchmarkXGBSE, BenchmarkPysurvival
#from pysurvival.models.survival_forest import ConditionalSurvivalForestModel

# setting seed
np.random.seed(42)

In [3]:
def split_train_test_valid(dataf, test_size=.2, valid_size=.01, random_state=1):
    df_train, df_test = train_test_split(dataf, test_size=test_size, random_state=random_state)
    df_train, df_valid = train_test_split(df_train, test_size=valid_size, random_state=random_state)
    return df_train, df_valid,  df_test

## Data

In [4]:
from pycox.datasets import rr_nl_nhp
df = rr_nl_nhp.read_df()
df.drop(["duration_true", "event_true", "censoring_true"], axis=1, inplace=True)
df_train, df_valid, df_test = split_train_test_valid(df)

T_train = df_train.duration
E_train = df_train.event
TIME_BINS = time_bins = get_time_bins(T_train, E_train, size=20)

Dataset 'rr_nl_nph' not created yet. Making dataset...
Done


## Model

Let us fit a model and check performance.

In [5]:
results = pd.DataFrame()

## Lifelines

In [6]:
bench_lifelines = BenchmarkLifelines(lifelines.CoxPHFitter(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "Cox-PH"
)

In [7]:
bench_lifelines.train()

In [8]:
cox_lifelines_results = bench_lifelines.test()
cox_lifelines_results

{'model': 'Cox-PH',
 'c-index': 0.7868823131406573,
 'ibs': 0.13529809371549895,
 'dcal_pval': 4.498777173708728e-43,
 'dcal_max_dev': 0.054902674947442935,
 'training_time': 2.267102003097534,
 'inference_time': 0.020849943161010742}

In [9]:
results = results.append(cox_lifelines_results, ignore_index=True)

In [10]:
bench_lifelines = BenchmarkLifelines(lifelines.WeibullAFTFitter(),
        df_train.assign(duration = np.where(df_train.duration == 0, 1, df_train.duration)),
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "Weibull AFT"
)

In [11]:
bench_lifelines.train()
weibull_aft_results = bench_lifelines.test()
weibull_aft_results

{'model': 'Weibull AFT',
 'c-index': 0.7868933380115156,
 'ibs': 0.13602422693625488,
 'dcal_pval': 2.8809253301697514e-43,
 'dcal_max_dev': 0.056672576269850755,
 'training_time': 0.32565784454345703,
 'inference_time': 0.009634017944335938}

In [12]:
results = results.append(weibull_aft_results, ignore_index=True)

## XGBoost

In [13]:
bench_xgboost = BenchmarkXGBoost(xgb,
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGB - AFT",
        "survival:aft",
)

In [14]:
bench_xgboost.train()
xgboost_aft_results = bench_xgboost.test()
xgboost_aft_results

{'model': 'XGB - AFT',
 'c-index': 0.8234768714689867,
 'ibs': nan,
 'dcal_pval': nan,
 'dcal_max_dev': nan,
 'training_time': 0.242509126663208,
 'inference_time': 0.0011980533599853516}

In [15]:
results = results.append(xgboost_aft_results, ignore_index=True)

In [16]:
bench_xgboost = BenchmarkXGBoost(xgb,
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGB - Cox",
        "survival:cox",
)
bench_xgboost.train()
xgboost_cox_results = bench_xgboost.test()
xgboost_cox_results

{'model': 'XGB - Cox',
 'c-index': 0.8235333028955971,
 'ibs': nan,
 'dcal_pval': nan,
 'dcal_max_dev': nan,
 'training_time': 0.37476611137390137,
 'inference_time': 0.001577138900756836}

In [17]:
results = results.append(xgboost_cox_results, ignore_index=True)

## XGB SE

In [18]:
bench_xgboost_embedding = BenchmarkXGBSE(XGBSEDebiasedBCE(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Debiased BCE",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_bce_results = bench_xgboost_embedding.test()
xgboost_bce_results

{'model': 'XGBSE - Debiased BCE',
 'c-index': 0.8242329843904878,
 'ibs': 0.10829539841030063,
 'dcal_pval': 2.3686030061958938e-135,
 'dcal_max_dev': 0.0683270751610485,
 'training_time': 4.561779975891113,
 'inference_time': 0.2850668430328369}

In [19]:
results = results.append(xgboost_bce_results, ignore_index=True)

In [20]:
bench_xgboost_embedding = BenchmarkXGBSE(XGBSEKaplanNeighbors(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Kaplan Neighbors",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_kn_results = bench_xgboost_embedding.test()
xgboost_kn_results

{'model': 'XGBSE - Kaplan Neighbors',
 'c-index': 0.8243235133558354,
 'ibs': 0.09968222364315912,
 'dcal_pval': 6.1562943589818576e-34,
 'dcal_max_dev': 0.037758403022887965,
 'training_time': 1.5040209293365479,
 'inference_time': 15.661872863769531}

In [21]:
results = results.append(xgboost_kn_results, ignore_index=True)

In [22]:
bench_xgboost_embedding = BenchmarkXGBSE(XGBSEKaplanTree(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Kaplan Tree",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_kt_results = bench_xgboost_embedding.test()
xgboost_kt_results

{'model': 'XGBSE - Kaplan Tree',
 'c-index': 0.82121853464918,
 'ibs': 0.10129775993937155,
 'dcal_pval': 6.905144620707361e-44,
 'dcal_max_dev': 0.043777519983996954,
 'training_time': 0.489609956741333,
 'inference_time': 0.006279945373535156}

In [23]:
results = results.append(xgboost_kt_results, ignore_index=True)

In [24]:
base_tree = XGBSEKaplanTree()

bench_xgboost_embedding = BenchmarkXGBSE(
    XGBSEBootstrapEstimator(base_tree, n_estimators=100),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Bootstrap Trees",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_bootstrap_results = bench_xgboost_embedding.test()
xgboost_bootstrap_results

{'model': 'XGBSE - Bootstrap Trees',
 'c-index': 0.8261708611753944,
 'ibs': 0.09713889680012697,
 'dcal_pval': 1.4238032769395418e-34,
 'dcal_max_dev': 0.035288583245685845,
 'training_time': 44.73573398590088,
 'inference_time': 0.5337009429931641}

In [25]:
results = results.append(xgboost_bootstrap_results, ignore_index=True)

In [26]:
bench_xgboost_embedding = BenchmarkXGBSE(XGBSEStackedWeibull(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Stacked Weibull",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_weibull_results = bench_xgboost_embedding.test()
xgboost_weibull_results

{'model': 'XGBSE - Stacked Weibull',
 'c-index': 0.8263715251908638,
 'ibs': 0.11344542750597256,
 'dcal_pval': 1.2434514782345034e-79,
 'dcal_max_dev': 0.05042037645542169,
 'training_time': 2.2547810077667236,
 'inference_time': 0.019007205963134766}

In [27]:
results = results.append(xgboost_weibull_results, ignore_index=True)

In [28]:
results.sort_values("c-index", ascending=False).round(3)

,c-index,dcal_max_dev,dcal_pval,ibs,inference_time,model,training_time
8,0.826,0.050,0.0,0.113,0.019,XGBSE - Stacked Weibull,2.255
7,0.826,0.035,0.0,0.097,0.534,XGBSE - Bootstrap Trees,44.736
5,0.824,0.038,0.0,0.100,15.662,XGBSE - Kaplan Neighbors,1.504
4,0.824,0.068,0.0,0.108,0.285,XGBSE - Debiased BCE,4.562
3,0.824,NaN,NaN,NaN,0.002,XGB - Cox,0.375
2,0.823,NaN,NaN,NaN,0.001,XGB - AFT,0.243
6,0.821,0.044,0.0,0.101,0.006,XGBSE - Kaplan Tree,0.490
1,0.787,0.057,0.0,0.136,0.010,Weibull AFT,0.326
0,0.787,0.055,0.0,0.135,0.021,Cox-PH,2.267


In [29]:
results.sort_values("c-index", ascending=False).round(3).set_index('model').to_markdown()

'| model                    |   c-index |   dcal_max_dev |   dcal_pval |     ibs |   inference_time |   training_time |\n|:-------------------------|----------:|---------------:|------------:|--------:|-----------------:|----------------:|\n| XGBSE - Stacked Weibull  |     0.826 |          0.05  |           0 |   0.113 |            0.019 |           2.255 |\n| XGBSE - Bootstrap Trees  |     0.826 |          0.035 |           0 |   0.097 |            0.534 |          44.736 |\n| XGBSE - Kaplan Neighbors |     0.824 |          0.038 |           0 |   0.1   |           15.662 |           1.504 |\n| XGBSE - Debiased BCE     |     0.824 |          0.068 |           0 |   0.108 |            0.285 |           4.562 |\n| XGB - Cox                |     0.824 |        nan     |         nan | nan     |            0.002 |           0.375 |\n| XGB - AFT                |     0.823 |        nan     |         nan | nan     |            0.001 |           0.243 |\n| XGBSE - Kaplan Tree      |     0.821 